<a href="https://colab.research.google.com/github/sahilmaniyar888/Neural-Network-from-Scratch/blob/main/Data_Loaders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
### Download Dogs vs Cats Zip ###
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

### Download IMDB Dataset ###
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

### Clean-Up CatsVDogs Images
!wget https://raw.githubusercontent.com/priyammaz/HAL-DL-From-Scratch/main/prep_data.py
!python -m prep_data --catsvdogs

--2026-02-01 12:33:43--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.0.194.53, 2600:1409:3c00:c8c::317f, 2600:1409:3c00:c80::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.0.194.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip’

kagglecatsanddogs_5 100%[===================>] 786.67M   274MB/s    in 2.9s    

2026-02-01 12:33:46 (274 MB/s) - ‘kagglecatsanddogs_5340.zip’ saved [824887076/824887076]

--2026-02-01 12:33:46--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclIm

## Images live on disk → paths are collected → labels are assigned → PyTorch samples by index → images are loaded one at a time when needed(lazy loading) .

In [9]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder # Stream data from images stored in folders

import os # Allows to access files
import numpy as np
from PIL import Image # Allows us to Load Images
from collections import Counter # Utility function to give us the counts of unique items in an iterable

In [10]:
path_to_folder = "/content/PetImages/"
os.listdir(path_to_folder)

['Cat', 'Dog']

In [11]:
import os
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms


class DogsVsCats(Dataset):

    def __init__(self, path_to_folder):
        # Build paths to class folders
        path_to_cats = os.path.join(path_to_folder, "Cat")
        path_to_dogs = os.path.join(path_to_folder, "Dog")

        # List all image filenames
        dog_files = os.listdir(path_to_dogs)
        cat_files = os.listdir(path_to_cats)

        # Build full paths
        path_to_dog_files = [
            os.path.join(path_to_dogs, file) for file in dog_files
        ]
        path_to_cat_files = [
            os.path.join(path_to_cats, file) for file in cat_files
        ]

        # Combine all image paths
        self.training_files = path_to_dog_files + path_to_cat_files

        # Labels
        self.dog_label = 0
        self.cat_label = 1

        # Image → Tensor transform
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.training_files)

    def __getitem__(self, idx):
        # Get image path
        path_to_image = self.training_files[idx]

        # Assign label based on folder name
        if "Dog" in path_to_image:
            label = self.dog_label
        else:
            label = self.cat_label

        # Load image and convert to tensor
        image = Image.open(path_to_image).convert("RGB")
        image = self.transform(image)

        return image, label


In [12]:
path_to_folder = "/content/PetImages"
dogvcat = DogsVsCats(path_to_folder)

print("Total Training Samples:", len(dogvcat))

image, label = dogvcat[0]
print("Label:", label)
print("Image Shape:", image.shape)


Total Training Samples: 24998
Label: 0
Image Shape: torch.Size([3, 500, 375])


In [13]:
### Create a Composition of Transformations
img_transforms = transforms.Compose(
    [
        transforms.Resize((224,224)), # Resize the image from whatever it is to a [3, 224, 224] image
        transforms.RandomHorizontalFlip(p=0.5), # Do a random flip with 50% probability
        transforms.ToTensor(), # Convert the image to a Tensor
        transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]) # Normalize the Image
    ]
)

### Lets Add this to our Dataset Class
class DogsVsCats(Dataset):
    def __init__(self, path_to_folder, transforms): ### Our INIT Now Accepts a Transforms

        ### PREVIOUS CODE ###
        path_to_cats = os.path.join(path_to_folder, "Cat")
        path_to_dogs = os.path.join(path_to_folder, "Dog")
        dog_files = os.listdir(path_to_dogs)
        cat_files = os.listdir(path_to_cats)
        path_to_dog_files = [os.path.join(path_to_dogs, file) for file in dog_files]
        path_to_cat_files = [os.path.join(path_to_cats, file) for file in cat_files]
        self.training_files = path_to_dog_files + path_to_cat_files
        self.dog_label, self.cat_label = 0, 1


        ### NEW CODE ###
        # self.transform = transforms.ToTensor() -> Notice how our transforms was just ToTensor before. It will be our Composition of Transforms now!
        self.transform = transforms

    def __len__(self):
        return len(self.training_files) # The number of samples we have is just the number of training files we have

    def __getitem__(self, idx):
        ### PREVIOUS CODE ###
        path_to_image = self.training_files[idx] # Grab file path at the sampled index
        if "Dog" in path_to_image: # If the word "Dog" is in the filepath, then set the label to 0
            label = self.dog_label
        else:
            label = self.cat_label # Otherwise set the label to 1
        image = Image.open(path_to_image) # Open Image with PIL to create a PIL Image

        ### UPDATED CODE ###
        image = self.transform(image) # Image now will go through series of transforms indicated in self.transform
        return image, label


### Instantiate Dataset With the Transforms ###
dogvcat = DogsVsCats(path_to_folder="PetImages/",
                     transforms=img_transforms)


dogsvcatsloader = DataLoader(dogvcat,
                             batch_size=16,
                             shuffle=True)

for images, labels in dogsvcatsloader:
    print(images.shape)
    print(labels)
    break

torch.Size([16, 3, 224, 224])
tensor([0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1])


In [14]:
train_samples = int(0.9 * len(dogvcat))
test_samples = len(dogvcat) - train_samples

print("Number of Training Samples:", train_samples, "Number of Test Samples", test_samples)

train_dataset, test_dataset = torch.utils.data.random_split(dogvcat, lengths=[train_samples, test_samples]) # Split with the dataset by the lenghts

### Load Datasets into two DataLoaders ###
trainloader = DataLoader(train_dataset,
                         batch_size=16,
                         shuffle=True)

testloader = DataLoader(test_dataset,
                        batch_size=16,
                        shuffle=True)


### Test Loaders ###
for images, labels in trainloader:
    print(images.shape)
    print(labels)
    break

for images, labels in testloader:
    print(images.shape)
    print(labels)
    break

Number of Training Samples: 22498 Number of Test Samples 2500
torch.Size([16, 3, 224, 224])
tensor([0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1])
torch.Size([16, 3, 224, 224])
tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0])
